In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import datetime
import logging
logger = logging.getLogger(__name__)
from kedro.framework.context import load_context
context = load_context('/Users/neil/Desktop/Hui_Yuan/Projects/pricing_airbnb-master', env='base')
catalog = context.catalog
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 300)
from functools import partial
from scipy.optimize import minimize_scalar
from prc.nodes.ds import embedding_model

/Users/neil/anaconda3/lib/python3.7/site-packages/kedro/config/config.py:274: UserWarning: Duplicate environment detected! Skipping re-loading from configuration path: /Users/neil/Desktop/Hui_Yuan/Projects/pricing_airbnb-master/conf/base
  "Skipping re-loading from configuration path: {}".format(item)
/Users/neil/anaconda3/lib/python3.7/site-packages/jinja2/utils.py:485: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/Users/neil/anaconda3/lib/python3.7/site-packages/kedro/config/config.py:274: UserWarning: Duplicate environment detected! Skipping re-loading from configuration path: /Users/neil/Desktop/Hui_Yuan/Projects/pricing_airbnb-master/conf/base
  "Skipping re-loading from configuration path: {}".format(item)
/Users/neil/anaconda3/lib/python3.7/site-packages/google/protobuf/descriptor.py:47: DeprecationWarning: Using or importing the ABCs fr

In [2]:
# step 0 : clean data
def load_data():
    df = catalog.load("fea_output")
    df = df.set_index(["listing_id", "date"])
    return df

In [3]:
df = load_data()
df.head()

2020-06-16 13:41:28,952 - kedro.io.data_catalog - INFO - Loading data from `fea_output` (CSVDataSet)...


available     price  minimum_nights  \
listing_id date                                             
8521       2019-05-23         f  5.298317        1.098612   
           2019-05-24         f  5.298317        1.098612   
           2019-05-25         f  5.298317        1.098612   
           2019-05-26         f  5.298317        1.098612   
           2019-05-27         t  5.298317        1.098612   

                       neighbourhood_group        room_type  \
listing_id date                                               
8521       2019-05-23                  NaN  Entire home/apt   
           2019-05-24                  NaN  Entire home/apt   
           2019-05-25                  NaN  Entire home/apt   
           2019-05-26                  NaN  Entire home/apt   
           2019-05-27                  NaN  Entire home/apt   

                       number_of_reviews  reviews_per_month  \
listing_id date                                               
8521       2019-05-23                 29               0.24   
           2019-05-24                 29               0.24   
           2019-05-25                 29               0.24   
           2019-05-26                 29               0.24   
           2019-05-27                 29               0.24   

                       calculated_host_listings_count  availability_365  \
listing_id date                                                           
8521       2019-05-23                               3          5.624018   
           2019-05-24                               3          5.624018   
           2019-05-25                               3          5.624018   
           2019-05-26                               3          5.624018   
           2019-05-27                               3          5.624018   

                            lag  month  
listing_id date                         
8521       2019-05-23  0.000000      5  
           2019-05-24  0.693147      5  
           2019-05-25  1.098612      5  
           2019-05-26  1.386294      5  
           2019-05-27  1.609438      5

In [6]:
# step 1: process data
def process_raw_data(df,
                     continuous_col = ["price", "minimum_nights", "lag", "number_of_reviews","calculated_host_listings_count"],
                     embedding_cols = ["neighbourhood_group", "room_type", "month"]):
    """
    
    :param df:
    :param continuous_col: ["price", "minimum_nights", "lag",...], the first element must be price.
    :param embedding_cols: the categorical data column
    :return:
    """
    df_select = df[continuous_col + embedding_cols + ["available"]]
    if df_select['neighbourhood_group'].isnull().sum()==len(df_select):
        df['neighbourhood_group']=np.ones((len(df['neighbourhood_group']))).astype(int)
    df_select = df_select.dropna(subset=embedding_cols + continuous_col, how="any")
    df_true = df_select[df_select["available"] == "t"]
    df_false = df_select[df_select["available"] != "t"]
    if len(df_true) < len(df_false):
        df_sampled = pd.concat([df_false.sample(n=len(df_true)), df_true], axis=0)
    else:
        df_sampled = pd.concat([df_false, df_true.sample(n=len(df_false))], axis=0)

    embed_lookup, x_embed_raw = embedding_model.build_embedding_dic(df_sampled, embedding_cols)
    return df_sampled, continuous_col, embedding_cols, embed_lookup, x_embed_raw

In [7]:
df_sampled, continuous_col, embedding_cols, embed_lookup, x_embed_raw = process_raw_data(df)
df_sampled.head()

,,price,minimum_nights,lag,number_of_reviews,calculated_host_listings_count,neighbourhood_group,room_type,month,available
listing_id,date,,,,,,,,,
15391083,2019-06-10,4.174387,0.000000,2.944439,162,2,1,Private room,6,f
19402921,2019-06-04,4.624973,1.609438,2.564949,0,1,1,Entire home/apt,6,f
30662073,2019-06-02,3.931826,0.693147,2.397895,20,3,1,Private room,6,f
14449525,2019-05-30,4.941642,0.693147,2.079442,133,1,1,Entire home/apt,5,f
3475883,2019-07-07,4.174387,1.098612,3.828641,33,4,1,Private room,7,f


In [8]:
# step 2: train_model
def train_model(df, continuous_col, embed_lookup, x_embed_raw, epochs):
    num_dimensions = len(continuous_col)
    num_samples = len(df)
    model = embedding_model.build_model(num_dimensions, num_samples, embed_lookup, lr=0.001)
    x_val = df[continuous_col].values
    X = [x_val, x_embed_raw[0], x_embed_raw[1], x_embed_raw[2]]
    y = (df["available"] == "t").astype(int)
    model.fit(X, y, epochs=epochs)
    # the parameter of w and bias
    candidate_w_bs = [(model.layers[-1].kernel_posterior.sample().numpy(),
                       model.layers[-1].bias_posterior.sample().numpy())]

    return model

In [9]:
model = train_model(df_sampled, continuous_col, embed_lookup, x_embed_raw, 10)


2020-06-16 13:42:27,318 - tensorflow - WARNING - From /Users/neil/anaconda3/lib/python3.7/site-packages/tensorflow_probability/python/layers/util.py:106: Layer.add_variable (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.add_weight` method instead.
Epoch 1/10


/Users/neil/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:349: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  if not isinstance(values, collections.Sequence):


1490/1490 [==============================] - 1s 937us/step - loss: 1.0785 - accuracy: 0.5164
Epoch 2/10
1490/1490 [==============================] - 1s 939us/step - loss: 0.8134 - accuracy: 0.5436
Epoch 3/10
1490/1490 [==============================] - 1s 941us/step - loss: 0.7311 - accuracy: 0.5611
Epoch 4/10
1490/1490 [==============================] - 1s 932us/step - loss: 0.6949 - accuracy: 0.5788
Epoch 5/10
1490/1490 [==============================] - 1s 917us/step - loss: 0.6761 - accuracy: 0.5888
Epoch 6/10
1490/1490 [==============================] - 1s 900us/step - loss: 0.6674 - accuracy: 0.5966
Epoch 7/10
1490/1490 [==============================] - 1s 941us/step - loss: 0.6605 - accuracy: 0.6061
Epoch 8/10
1490/1490 [==============================] - 1s 903us/step - loss: 0.6567 - accuracy: 0.6063
Epoch 9/10
1490/1490 [==============================] - 1s 913us/step - loss: 0.6533 - accuracy: 0.6112
Epoch 10/10
1490/1490 [==============================] - 1s 936us/step - lo

In [36]:
# step3.1 deal with data
def series_row(row, embedding_cols, continuous_col, embed_lookup) :
    
    continuous = np.array(row[continuous_col].values.tolist()).reshape(-1, len(continuous_col))
    input_row = [continuous]
    for key,val in embed_lookup.items():
        input_row.append(np.array([val[row[key]]]))
    return input_row
    

# step3.2 calc revenue
def cal_neg_revnue(price, input_model, row, embed_lookup, continuous_col):
    """ only change price to figure out the accept rate """
    continuous = np.append(np.array(price),row[0][0][1:]).reshape(-1,len(continuous_col))
    new_price_row = [continuous]
    for i in range(1,len(row)):
        new_price_row.append(row[i])
    prob = input_model.predict(new_price_row)
    return -1 * price * prob[0][0]


# step3.3 estimate price and prob
def optimal_sellable_capacity(model=None, row=None, method="Bounded",
                              min_search_bound=1, max_search_bound=10,
                              embed_lookup=None, continuous_col=None):
    func = partial(
        cal_neg_revnue,
        input_model=model,
        row=row,
        embed_lookup=embed_lookup,
        continuous_col=continuous_col

    )

    min_ = minimize_scalar(
        func, bounds=(min_search_bound, max_search_bound), method=method
    )

    best_price = min_.x

    x0 = np.append(np.array(best_price), row[0][0][1:]).reshape(-1, len(continuous_col))
    new_price_row = [x0]
    for i in range(1, len(row)):
        new_price_row.append(np.array(row[i]))
    prob = model.predict(new_price_row)
    return best_price, prob[0][0]

# step3.4 combined
def wrapper_price_func(model=None, row=None, continuous_col=None, embed_lookup= None,
                       min_search_bound=1, max_search_bound=10):
    input_row = series_row(row, embedding_cols, continuous_col, embed_lookup)
    est_price, prob = optimal_sellable_capacity(model=model, row=input_row,
                                                min_search_bound=min_search_bound,
                                                max_search_bound=max_search_bound,
                                                continuous_col=continuous_col,
                                                embed_lookup=embed_lookup)
    return est_price, prob


# step 3.5 apply to dataframe
def run_estimate_price(df,model,continuous_col,embedding_cols, embed_lookup):
    
    
    df_sampled_new = df.reset_index(level=['listing_id','date'])
    index_series = df_sampled_new[['listing_id','date']]
    df_sampled_new = df_sampled_new[continuous_col + embedding_cols + ['listing_id','date']] 
    
    
    ### select sample to test 
    ### date:'2019-05-01' to '2019-6-01' inclusive
    ### test first 20 rows
    sample_num = 20
    df_sampled_new = df_sampled_new[df_sampled_new['date'].between('2019-05-01', '2019-6-01', inclusive=True)]
    df_new = df_sampled_new.iloc[:sample_num, :-2].reset_index(drop=True)
    # estimate best rental price 
    df_new['price_prob'] = df_test.apply(lambda x: wrapper_price_func(min_search_bound=1,max_search_bound=10,
                                                continuous_col=continuous_col, embed_lookup= embed_lookup,
                                                model= model, row = x)
                                        , axis=1)
    df_new['estimate_price'] = df_new['price_prob'].apply(lambda x: '$' + str(round(np.exp(x[0]), 2)))
    df_new['prob'] = df_new['price_prob'].apply(lambda x: str(round(x[1] * 100, 2)) + '%')
    df_new = df_new.drop("price_prob", axis=1)
    
    # get back log columns
    log_cols = ["price", "minimum_nights", "lag"]
    df_new[log_cols] = np.exp(df_new[log_cols])

    # change price, minimum_nights, lag format
    df_new['price'] = df_new['price'].apply(lambda x: '$' + str(round(x, 2)))
    df_new['minimum_nights'] = df_new['minimum_nights'].apply(lambda x: int(x))
    df_new['lag'] = df_new['lag'].apply(lambda x: int(x))

    # reset index
    index_new_listId = index_series.iloc[:sample_num, 0]
    index_new_date = index_series.iloc[:sample_num, 1]
    df_new = df_new.set_index([index_new_listId, index_new_date])
    return df_new



In [37]:
df_new = run_estimate_price(df,model,continuous_col,embedding_cols, embed_lookup)
df_new

price  minimum_nights  lag  number_of_reviews  \
listing_id date                                                         
8521       2019-05-23  $200.0               3    1                 29   
           2019-05-24  $200.0               3    2                 29   
           2019-05-25  $200.0               3    3                 29   
           2019-05-26  $200.0               3    4                 29   
           2019-05-27  $200.0               3    5                 29   
           2019-05-28  $200.0               3    6                 29   
           2019-05-29  $400.0               3    6                 29   
           2019-05-30  $400.0               3    8                 29   
           2019-05-31  $400.0               3    9                 29   
           2019-06-01  $400.0               3   10                 29   
           2019-06-02  $200.0               3   11                 29   
           2019-06-03  $200.0               3   12                 29   
           2019-06-04  $200.0               3   13                 29   
           2019-06-05  $200.0               3   13                 29   
           2019-06-06  $200.0               3   14                 29   
           2019-06-07  $200.0               3   15                 29   
           2019-06-08  $200.0               3   17                 29   
           2019-06-09  $200.0               3   17                 29   
           2019-06-10  $200.0               3   18                 29   
           2019-06-11  $200.0               3   20                 29   

                       calculated_host_listings_count  neighbourhood_group  \
listing_id date                                                              
8521       2019-05-23                               3                    1   
           2019-05-24                               3                    1   
           2019-05-25                               3                    1   
           2019-05-26                               3                    1   
           2019-05-27                               3                    1   
           2019-05-28                               3                    1   
           2019-05-29                               3                    1   
           2019-05-30                               3                    1   
           2019-05-31                               3                    1   
           2019-06-01                               3                    1   
           2019-06-02                               3                    1   
           2019-06-03                               3                    1   
           2019-06-04                               3                    1   
           2019-06-05                               3                    1   
           2019-06-06                               3                    1   
           2019-06-07                               3                    1   
           2019-06-08                               3                    1   
           2019-06-09                               3                    1   
           2019-06-10                               3                    1   
           2019-06-11                               3                    1   

                             room_type  month estimate_price    prob  
listing_id date                                                       
8521       2019-05-23  Entire home/apt      5      $21435.83  93.49%  
           2019-05-24  Entire home/apt      5      $21019.74  92.62%  
           2019-05-25  Entire home/apt      5       $18186.3  92.41%  
           2019-05-26  Entire home/apt      5      $20730.06  94.18%  
           2019-05-27  Entire home/apt      5      $21601.92  94.45%  
           2019-05-28  Entire home/apt      5      $21752.77  91.84%  
           2019-05-29  Entire home/apt      5      $18977.57  93.51%  
           2019-05-30  Entire home/apt      5      $18619.68 

In [ ]:
continuous_col = ["price", "minimum_nights", "lag", "number_of_reviews","calculated_host_listings_count"]
embedding_cols = ["neighbourhood_group", "room_type", "month"]